In [152]:
import os
import pandas as pd
import geopandas as gpd

import logging
logging.getLogger().setLevel(logging.INFO) # DEBUG, INFO or WARN
DROOT = '../1-data/'

In [138]:
# Loading this file might take a moment.
urbancenters = gpd.read_file(
    os.path.join(DROOT, '2-external', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A', 'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'))
urbancenters.head(2)

ID_HDC_G0  QA2_1V   AREA  BBX_LATMN   BBX_LONMN  BBX_LATMX   BBX_LONMX  \
0        1.0     1.0  185.0  21.247683 -158.043016  21.422193 -157.730529   
1        2.0     2.0   42.0 -17.641184 -149.628088 -17.517631 -149.508018   

   GCPNT_LAT   GCPNT_LON         CTR_MN_NM  ...      EX_SS_P00      EX_SS_P15  \
0  21.340678 -157.893497     United States  ...  397443.031445  444041.529529   
1 -17.534103 -149.568053  French Polynesia  ...       0.000000       0.000000   

   EX_EQ19PGA EX_EQ19MMI EX_EQ19_Q EX_HW_IDX SDG_LUE9015 SDG_A2G14 SDG_OS15MX  \
0         NaN        NaN   missing       NaN    0.074385  0.226415      56.41   
1         NaN        NaN   missing       NaN       0.128  0.284119        NaN   

                                            geometry  
0  MULTIPOLYGON (((-158.01244 21.42219, -157.9915...  
1  MULTIPOLYGON (((-149.56967 -17.51763, -149.508...  

[2 rows x 161 columns]

In [180]:
cities = pd.read_excel(os.path.join(DROOT, '1-research', 'cities.latest.xlsx'))
cities.sort_values('priority').head(2)

city_name country_id  city_id  priority
22        São Paulo        BRA     1303        -1
0   s-Hertogenbosch        NLD     2194         0

In [187]:
# Initiate clean output list to add to excel.
cities_out = []

In [194]:
# Actually search cities here: 
city_name = "arcel"
search = urbancenters[urbancenters.UC_NM_LST.str.contains(city_name)]
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'UC_NM_LST', 'AREA']]

ID_HDC_G0   UC_NM_MN CTR_MN_ISO  \
903       904.0  Barcelona        VEN   
1451     1452.0      Dakar        SEN   
2050     2051.0  Barcelona        ESP   

                                              UC_NM_LST   AREA  
903                 Barcelona; Puerto La Cruz; Lechería  145.0  
1451  Dakar; Guédiawaye; Parcelles Assainies; Rufisq...  277.0  
2050  Barcelona; l'Hospitalet de Llobregat; Badalona...  583.0

In [188]:
# Pass the ID_HDC_G0 in here below:
ID_HDC_G0 = 2083.0
record = urbancenters[urbancenters.ID_HDC_G0 == ID_HDC_G0]
cities_out.append(record[['ID_HDC_G0', 'CTR_MN_ISO', 'UC_NM_MN']].iloc[0].to_list())
record

ID_HDC_G0  QA2_1V  AREA  BBX_LATMN  BBX_LONMN  BBX_LATMX  BBX_LONMX  \
2082     2083.0     1.0  29.0  41.948419   2.770985  42.029851   2.843465   

      GCPNT_LAT  GCPNT_LON CTR_MN_NM  ... EX_SS_P00  EX_SS_P15  EX_EQ19PGA  \
2082  41.982266   2.809519     Spain  ...       0.0        0.0    0.041391   

     EX_EQ19MMI  EX_EQ19_Q EX_HW_IDX SDG_LUE9015 SDG_A2G14 SDG_OS15MX  \
2082        4.0  available   20.5156     0.35298  0.132259      56.03   

                                               geometry  
2082  MULTIPOLYGON (((2.82098 42.02985, 2.83298 42.0...  

[1 rows x 161 columns]

In [195]:
cities_new = pd.DataFrame(cities_out, columns=['city_id', 'country_id', 'city_name'])
cities_new['priority'] = 0
cities_new

city_id country_id city_name  priority
0   2083.0        ESP    Girona         0

In [197]:
# Write out to file
cities_v2 = pd.concat([cities, cities_new])
cities_v2 = cities_v2.groupby('city_id').apply(lambda x: x.sort_values('priority', ascending=True).iloc[0])
cities_v2 = cities_v2.sort_values(['priority', 'country_id', 'city_name'])
cities_v2.to_excel('../1-data/1-research/cities.latest.xlsx', index=False)
cities_v2.head(2)

city_name country_id  city_id  priority
city_id                                         
1303.0   São Paulo        BRA   1303.0        -1
2083.0      Girona        ESP   2083.0         0

In [ ]:
search = urbancenters[urbancenters.CTR_MN_ISO == 'NLD']
search[['ID_HDC_G0', 'UC_NM_MN', 'CTR_MN_ISO', 'AREA']].sort_values('AREA', ascending=False)

ID_HDC_G0               UC_NM_MN CTR_MN_ISO   AREA
2122     2123.0  Rotterdam [The Hague]        NLD  658.0
2166     2167.0              Amsterdam        NLD  345.0
2234     2235.0                Heerlen        NLD  129.0
2180     2181.0                Utrecht        NLD  124.0
2204     2205.0              Eindhoven        NLD  104.0
2137     2138.0                 Leiden        NLD   97.0
2162     2163.0              Dordrecht        NLD   88.0
2225     2226.0                 Arnhem        NLD   87.0
2182     2183.0                Tilburg        NLD   63.0
2143     2144.0             Zoetermeer        NLD   60.0
2218     2219.0               Nijmegen        NLD   60.0
2165     2166.0                Alkmaar        NLD   59.0
2186     2187.0                 Almere        NLD   56.0
2151     2152.0              Heemskerk        NLD   54.0
2154     2155.0                Haarlem        NLD   52.0
2168     2169.0                  Breda        NLD   48.0
2193     2194.0       's-Hertogenbosch        NLD   45.0
2268     2269.0              Groningen        NLD   44.0
2297     2298.0               Enschede        NLD   43.0
2198     2199.0             Amersfoort        NLD   41.0
2219     2220.0             Maastricht        NLD   41.0
2228     2229.0              Apeldoorn        NLD   39.0
2232     2233.0                 Zwolle        NLD   36.0
2291     2292.0                Hengelo        NLD   35.0
2278     2279.0                 Almelo        NLD   34.0
2246     2247.0                  Venlo        NLD   32.0
2226     2227.0                Sittard        NLD   29.0
2163     2164.0                  Gouda        NLD   29.0
2243     2244.0               Deventer        NLD   27.0
2178     2179.0                  Hoorn        NLD   25.0
2216     2217.0             Leeuwarden        NLD   24.0
2217     2218.0                Helmond        NLD   23.0
2175     2176.0              Purmerend        NLD   21.0
2161     2162.0    Alphen aan den Rijn        NLD   20.0
2144     2145.0             Roosendaal        NLD   20.0
2208     2209.0             Veenendaal        NLD   18.0
2187     2188.0              Hilversum        NLD   17.0